In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd

In [2]:
credentials = json.load(open('credentials.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

In [3]:
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

** LOAD IN CSV **

In [5]:
songs_0 = pd.read_csv('data/Songs/songs0.csv')

In [6]:
playlist_one = songs_0[songs_0['pid'] == 0]

In [7]:
track_uri = playlist_one['track_uri']

In [9]:
uri_set = set()
for item in track_uri:
    uri_set.add(item)

In [10]:
uri_set

{'0UaMYEvWZi0ZqiDOoHU3YI',
 '0WqIKmW4BTrj3eJFmnCKMv',
 '0XUfyU2QviPAs6bxSpXYG4',
 '0uqPG793dkDDN7sCUJJIVC',
 '12qZHAeOyTf93YAWvGDTat',
 '19Js5ypV6JKn4DMExHQbGc',
 '1AWQoqb9bSvzTjaLralEkT',
 '1Bv0Yl01xBDZD4OQP93fyl',
 '1HwpWwa6bnqqRhK8agG4RS',
 '1JURww012QnWAw0zZXi6Aa',
 '1TfAhjzRBWzYZ8IdUV3igl',
 '1X5WGCrUMuwRFuYU1eAo2I',
 '1Y4ZdPOOgCUhBcKZOrUFiS',
 '1b7vg5T9YKR3NNqXfBYRF7',
 '1kusepF3AacIEtUTYrw4GV',
 '1lzr43nnXAijIGYnCT8M8H',
 '20ORwCJusz4KS2PbTPVNKo',
 '215JYyyUnrJ98NK3KEwu6d',
 '2PpruBYCo4H7WOBJ7Q2EwM',
 '2eJ8ij1T3cNUKiGdcUvKhy',
 '2gam98EZKrF9XuOkU13ApN',
 '2jFlMILIQzs7lSFudG9lbo',
 '2nbClS09zsIAqNkshg6jnp',
 '34ceTg8ChN5HjrqiIYCn9Q',
 '3BxWKCI06eQ5Od8TY2JBeA',
 '3H1LCvO3fVsK2HPguhbml0',
 '3jkdQNkDTxxXtjSO4l0o1H',
 '3uoQULcUWfnt6nc6J7Vgai',
 '3utIAb67sOu0QHxBE88P1M',
 '4E5P1XyAFtrjpiIxkydly4',
 '4I2GqMe7L2ccMpUbnDzYLH',
 '4Y45aqo9QMa57rDsAJv40A',
 '4omisSlTk6Dsq2iQD7MA07',
 '4pmc2AxSEq6g7hPVlJCPyP',
 '5Q0Nhxo0l2bP3pNjpGJwV1',
 '5c1sfI6wIQEsSUw0xrkFdl',
 '5lDriBxJd22IhOH9zTcFrV',
 

In [49]:
dict_list = []
for uri in uri_set:
    track_object = sp.track('0UaMYEvWZi0ZqiDOoHU3YI')
    album_type = track_object['album']['album_type']
    if album_type != 'single':
        track_id = track_object['id']
        album_id = track_object['album']['id']
        track_count = track_object['album']['total_tracks']
        duration = track_object['duration_ms']
        explicit = track_object['explicit']
        track_pop = track_object['popularity']
        track_number = track_object['track_number']

        album = sp.album(album_id)
        album_pop = album['popularity']
        comparative_pop = track_pop - (((album_pop * track_count) - track_pop)/ (track_count - 1))

        features = sp.audio_features(track_id)
        danceability = features[0]['danceability']
        energy = features[0]['energy']
        loudness = features[0]['loudness']
        speechiness = features[0]['speechiness']
        acousticness = features[0]['acousticness']
        instrumentalness = features[0]['instrumentalness']
        liveness = features[0]['liveness']
        valence = features[0]['valence']
        tempo = features[0]['tempo']  

        mydict = {'track_id': track_id, 'album_id' : album_id, 'track_number': track_number, 
                        'track_count': track_count, 'duration': duration, 'explicit': explicit, 
                        'track_pop': track_pop,'album_pop': album_pop, 'comparative_pop': comparative_pop, 
                        'danceability': danceability, 'energy': energy, 'loudness': loudness, 
                        'speechiness': speechiness, 'acousticness': acousticness, 'intstrumentalness': instrumentalness, 
                        'liveness': liveness, 'valence': valence, 'tempo': tempo}
        
        dict_list.append(mydict)

retrying ...1secs


In [50]:
dict_list

[{'track_id': '0UaMYEvWZi0ZqiDOoHU3YI',
  'album_id': '6vV5UrXcfyQD1wu4Qo2I9K',
  'track_number': 4,
  'track_count': 16,
  'duration': 226863,
  'explicit': True,
  'track_pop': 68,
  'album_pop': 59,
  'comparative_pop': 9.600000000000001,
  'danceability': 0.904,
  'energy': 0.813,
  'loudness': -7.105,
  'speechiness': 0.121,
  'acousticness': 0.0311,
  'intstrumentalness': 0.00697,
  'liveness': 0.0471,
  'valence': 0.81,
  'tempo': 125.461},
 {'track_id': '0UaMYEvWZi0ZqiDOoHU3YI',
  'album_id': '6vV5UrXcfyQD1wu4Qo2I9K',
  'track_number': 4,
  'track_count': 16,
  'duration': 226863,
  'explicit': True,
  'track_pop': 68,
  'album_pop': 59,
  'comparative_pop': 9.600000000000001,
  'danceability': 0.904,
  'energy': 0.813,
  'loudness': -7.105,
  'speechiness': 0.121,
  'acousticness': 0.0311,
  'intstrumentalness': 0.00697,
  'liveness': 0.0471,
  'valence': 0.81,
  'tempo': 125.461},
 {'track_id': '0UaMYEvWZi0ZqiDOoHU3YI',
  'album_id': '6vV5UrXcfyQD1wu4Qo2I9K',
  'track_number'

In [38]:
album = sp.album('6vV5UrXcfyQD1wu4Qo2I9K')
album['popularity']

59

In [ ]:
one_track = sp.track('0UaMYEvWZi0ZqiDOoHU3YI')

In [25]:
one_track

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2wIVse2owClT7go1WT98tk'},
    'href': 'https://api.spotify.com/v1/artists/2wIVse2owClT7go1WT98tk',
    'id': '2wIVse2owClT7go1WT98tk',
    'name': 'Missy Elliott',
    'type': 'artist',
    'uri': 'spotify:artist:2wIVse2owClT7go1WT98tk'}],
  'available_markets': ['AD',
   'AE',
   'AR',
   'AT',
   'AU',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HU',
   'ID',
   'IE',
   'IL',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PS',
   'PT',
   'PY',
   'QA',
   'RO',
   'SA',
   'SE',
   'SG',
   'SK',
   'SV'

In [23]:
one_album = one_track['album']

In [26]:
one_album['album_type'] != 'single'

'album'

In [ ]:
#

In [18]:
track_uri[0]

'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI'

In [40]:
features = sp.audio_features('0UaMYEvWZi0ZqiDOoHU3YI')

In [41]:
features

[{'danceability': 0.904,
  'energy': 0.813,
  'key': 4,
  'loudness': -7.105,
  'mode': 0,
  'speechiness': 0.121,
  'acousticness': 0.0311,
  'instrumentalness': 0.00697,
  'liveness': 0.0471,
  'valence': 0.81,
  'tempo': 125.461,
  'type': 'audio_features',
  'id': '0UaMYEvWZi0ZqiDOoHU3YI',
  'uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
  'track_href': 'https://api.spotify.com/v1/tracks/0UaMYEvWZi0ZqiDOoHU3YI',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0UaMYEvWZi0ZqiDOoHU3YI',
  'duration_ms': 226864,
  'time_signature': 4}]